[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ishandandekar/Looking-Fruit/blob/main/Looking_Fruit_nbk.ipynb)

# Looking_Fruit

👋 Hello and welcome to the **Looking_Fruit** notebook. In this notebook I try to replicate the [Fruits-360](https://www.researchgate.net/publication/321475443_Fruit_recognition_from_images_using_deep_learning) research paper. In this paper, researchers have tried to classify images of **131** fruits and vegetables. The data used for these modelling experiments is provided by the paper researchers themselves.

In [ ]:
# Check for GPU
!nvidia-smi -L

## Step 0: Defining the problem


**Objective:**  
To classify the images of various fruits and vegetables with best f1-score.  

**Files:**
- *Train*: This folder contains folders labelled as fruit's/vegetable's name. These subfolders contain images of the respective fruit/vegetable. This folder will be used for training purpose.
- *Test*: This folder contains folders labelled as fruit's/vegetable's name. These subfolders contain images of the respective fruit/vegetable. This folder will be used for testing purpose.

## Step 1: Getting the data
The data used for this project is publicly available on [Kaggle](https://www.kaggle.com/datasets/ishandandekar/fruitimagedataset).

- Use Kaggle's API to download the data into Colab.
- Get utility functions to help in future.
- Configure data files to read using Python.


In [ ]:
# Getting the helper functions script
!wget https://raw.githubusercontent.com/ishandandekar/Looking-Fruit/main/helper_functions.py

# Get the necessary functions from the python script
from helper_functions import plot_loss_curves, unzip_data, create_model_checkpoint, get_metrics

About these helper functions (see documentation for better description):
* **plot_loss_curves** : Plots a line graph (using matplotlib) to see changes in loss and accuracy during training.
* **unzip_data** : Unzips a zip file to a directory
* **create_model_checkpoint** : Create a model checkpoint with only best weights (monitored over validation loss).

In [ ]:
# Install the kaggle library
!pip install -q kaggle

# Upload the Kaggle API keys
from google.colab import files
files.upload()

!mkdir ~/.kaggle

# Copy the json file to the folder
!cp kaggle.json ~/.kaggle

# Change permissions for json to work with the Kaggle API
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d ishandandekar/fruitimagedataset

# Unzip data
unzip_data('fruitimagedataset.zip', data_dir="raw")

## Step 2: Know more about the data

- Get the statistics about the data.
- Check if the labels are imbalanced.
- Visualize random samples in data.
- (*If required*) Trim data.
- (*If required*) Preprocess the data.
- Make data processing faster using `ImageDataGenerator`.

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from keras.utils import plot_model

In [ ]:
# Setting up necessary variables
TRAIN_PATH = '/content/raw/train/train/'
TEST_PATH = '/content/raw/test/test/'
RAW_DATA_PATH = '/content/raw'

In [ ]:
# Checking the number of classes
classes = []

for dirname, _, filenames in os.walk(RAW_DATA_PATH):
    if dirname.startswith(f"{TRAIN_PATH}"):
        classes.append(dirname[len(TRAIN_PATH):])

print(f"Total number of classes to deal with: {len(classes)}")

In [ ]:
# Show random classes present in the dataset
list_of_five_random_labels = random.sample(classes,5)
list_of_five_random_labels

In [ ]:
# Number of images of each fruit/vegetable as a pandas.DataFrame

# List to append the count of images
number_of_images_train = []

for label in classes:
    path = f'{TRAIN_PATH}/{label}'
    count = len(os.listdir(path))
    number_of_images_train.append(count)

train_image_count_df = pd.DataFrame({"Label":classes,"Number of Images":number_of_images_train})

# To view first 10 rows of the dataframe
train_image_count_df.head(10)

In [ ]:
# Label with most number of images
print(f"Label with most number of images:")
print(train_image_count_df.sort_values("Number of Images",ascending=False).head(1))
print(f"Label with least number of images:")
print(train_image_count_df.sort_values("Number of Images",ascending=True).head(1))

In [ ]:
# Show random sample from training of a random fruit/vegetable

random_label = random.choice(classes)
sample_path = f'{TRAIN_PATH}/{random_label}'
random_image= random.choice(os.listdir(sample_path))
random_image_path = f'{sample_path}/{random_image}'

img = mpimg.imread(random_image_path)
imgplot = plt.imshow(img)
plt.axis(False)
plt.title(f'{random_label}')
plt.show()

In [ ]:
# Making new directories for trimmed training data
os.mkdir("data")
os.mkdir("data/train")

In [ ]:
# Trimming data as there are lot of training examples

TRAIN_IMAGES_COUNT = 100

for label in classes:
  path = f'{TRAIN_PATH}/{label}'
  img_files = os.listdir(path)
  random_count_images = random.sample(img_files, TRAIN_IMAGES_COUNT)
  new_path_for_label = f"/content/data/train/{label}"
  os.mkdir(new_path_for_label)

  for img_file in random_count_images:
    shutil.move(f"{path}/{img_file}", new_path_for_label)

In [ ]:
# Image size has been specified in the research paper
IMAGE_SIZE = (100,100)
BATCH_SIZE = 32
TRIMMED_TRAIN_PATH = '/content/data/train'
TEST_PATH = '/content/raw/test/test'

# To make data processing faster
train_data=tf.keras.preprocessing.image_dataset_from_directory(directory=TRIMMED_TRAIN_PATH,
                                                               image_size=IMAGE_SIZE,
                                                               label_mode="categorical",
                                                               batch_size=BATCH_SIZE)

test_data=tf.keras.preprocessing.image_dataset_from_directory(directory=TEST_PATH,
                                                               image_size=IMAGE_SIZE,
                                                               label_mode="categorical",
                                                               batch_size=BATCH_SIZE)

In [ ]:
train_data

## Step 3: Describing modelling experiments

- This notebook contains 7 models built to get the best **f1-score** on the test dataset. These models also include the models made by the researchers themselves.  
- Models to be made:
  1. **Model 0** : A simple model with fully connected multiple Dense layers; this model acts as a baseline.
  1. **Model 1** : Multiple pairs of CNN and MaxPool layers with a  Dense layer in the end for classification.
  1. **Model 2** : Replicate the best model (on the test set) from research paper; *should get better results from this.*
  1. **Model 3** : Using transfer learning, exploit ResNet model for classification.
  1. **Model 4** : Using transfer learning, exploit EfficientNetBx for classification.
  1. **Model 5** : Use fine-tuned ResNet model for classification.
  1. **Model 6** : Use fine-tuned EfficientNetBx for classification.
- Get classification metrics for each model.


In [ ]:
# Directory to save images of models' architectures
!mkdir model_architectures

#### Model 0 
* **Layers:** 3 Dense layers
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 5
* **Callbacks:** `ModelCheckpoint`
* **Validation data:** available

In [ ]:
# 0. Set random seed
tf.random.set_seed(42)
INPUT_SHAPE = (100,100,3)

# 1. Create the model
model_0 = tf.keras.Sequential([
    layers.Flatten(input_shape=(100,100,3)),
    layers.Input(shape=INPUT_SHAPE),
    layers.Dense(128, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(131, activation="softmax")
], name="model_0_dense")

# 2. Compile the model
model_0.compile(loss="categorical_crossentropy",
                optimizer="Adam",
                metrics=["accuracy"])

# 3. Get the summary
model_0.summary()

# 4. Fit the model
history_model_0 = model_0.fit(train_data,
                              epochs=10,
                              validation_data=test_data,
                              validation_steps=len(test_data)*0.15,
                              callbacks=[create_model_checkpoint(model_name=model_0.name)])

In [ ]:
# Save model's architecture as an image
plot_model(model_0, to_file='/content/model_architectures/model_0.png')

# About the model history
plot_loss_curves(history_model_0)

In [ ]:
# Loading in the best weights
best_weights_path = "/content/model_experiments/model_0_dense"
model_0.load_weights(best_weights_path)

# Evaluate on test set
model_0.evaluate(test_data)

In [ ]:
# Unravel dataset
y_labels = []
for images,labels in test_data.unbatch():
  y_labels.append(labels.numpy().argmax())

In [ ]:
# Getting predictions on test data
model_0_preds_prob = model_0.predict(test_data, verbose=1)
model_0_preds_classes = model_0_preds_prob.argmax(axis=1)

# Getting a metrics report
model_0_metrics = get_metrics(y_labels, model_0_preds_classes)
model_0_metrics

#### Model 1 
* **Layers:** Multiple Conv2D layers paired with MaxPool layers with a Dense layer at the end
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 10
* **Callbacks:** `ModelCheckpoint`
* **Validation data:** available

In [ ]:
# 0. Set random seed
tf.random.set_seed(42)
INPUT_SHAPE = (100,100,3)

# 1. Create the model
model_1 = tf.keras.Sequential([
     tf.keras.layers.Conv2D(filters=10,
                            kernel_size=3,
                            activation="relu",
                            input_shape=INPUT_SHAPE),
     tf.keras.layers.Conv2D(10,3,activation="relu"),
     tf.keras.layers.MaxPool2D(pool_size=2,
                               padding="valid"),
     tf.keras.layers.Conv2D(10,3,activation="relu"),
     tf.keras.layers.Conv2D(10,3,activation="relu"),
     tf.keras.layers.MaxPool2D(2),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(131,activation="softmax")
], name="model_1_Conv2D")

# 2. Compile the model
model_1.compile(loss="categorical_crossentropy",
                optimizer="Adam",
                metrics=["accuracy"])

# 3. Get the summary
model_1.summary()

# 4. Fit the model
history_model_1 = model_1.fit(train_data,
                              epochs=10,
                              validation_data=test_data,
                              validation_steps=len(test_data)*0.15,
                              callbacks=[create_model_checkpoint(model_name=model_0.name)])

In [ ]:
# About the model history
plot_loss_curves(history_model_1)

In [ ]:
# Loading in the best weights
best_weights_path = "/content/model_experiments/model_1_Conv2D"
model_1.load_weights(best_weights_path)

# Evaluate on test set
model_1.evaluate(test_data)

In [ ]:
# Getting predictions on test data
model_1_preds_prob = model_1.predict(test_data, verbose=1)
model_1_preds_classes = model_1_preds_prob.argmax(axis=1)

# Getting a metrics report
model_1_metrics = get_metrics(y_labels, model_1_preds_classes)
model_1_metrics

#### Model 2 
* **Layers:** 3 Dense layers
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 5
* **Callbacks:** `ModelCheckpoint`
* **Validation data:** available

#### Model 3
* **Layers:** 3 Dense layers
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 5
* **Callbacks:** `ModelCheckpoint`
* **Validation data:** available

#### Model 4 
* **Layers:** 3 Dense layers
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 5
* **Callbacks:** `ModelCheckpoint`
* **Validation data:** available

#### Model 5
* **Layers:** 3 Dense layers
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 5
* **Callbacks:** `ModelCheckpoint`
* **Validation data:** available

#### Model 6
* **Layers:** 3 Dense layers
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 5
* **Callbacks:** `ModelCheckpoint`
* **Validation data:** available

## Step 4: Compare results and conclude experiments
- Test each model on the given test dataset.
- Use graphs and matrices to visualize results.
- (*Optional*) Tune hyperparameters of the best model.
- Compare best models results with researchers best model.
- Export the best model.
